# 21-1 py-zerox를 활용한 데이터 전처리

In [ ]:
import nest_asyncio
nest_asyncio.apply()
from dotenv import load_dotenv
from pyzerox import zerox
import asyncio

load_dotenv()

# OpenAI 모델
model = "gpt-4o"

async def main():
    file_path = "./income_tax.pdf"  # 로컬 파일 경로 및 파일 URL 지원

    select_pages = None  # 모든 페이지를 처리하려면 None, 특정 페이지 번호(1부터 시작)를 처리하려면 int 또는 list(int)

    output_dir = "./output"  # 통합된 마크다운 파일을 저장할 디렉터리
    result = await zerox(file_path=file_path, model=model, output_dir=output_dir, select_pages=select_pages)
    return result

# 메인 함수 실행
result = asyncio.run(main())

# 마크다운 결과 출력
print(result)


In [14]:
import nest_asyncio
nest_asyncio.apply()

In [21]:
import markdown
from bs4 import BeautifulSoup
markdown_path = './output/income_tax.md'
text_path = './output/income_tax.txt'

# 마크다운 파일을 읽어온다.
with open(markdown_path, 'r', encoding='utf-8') as md_file:
    md_content = md_file.read()

# 마크다운 콘텐츠를 HTML로 변환한다.
html_content = markdown.markdown(md_content)

# HTML 콘텐츠를 파싱하여 텍스트만 추출한다.
soup = BeautifulSoup(html_content, 'html.parser')
text_content = soup.get_text()

# 추출한 텍스트를 텍스트 파일로 저장한다.
with open(text_path, 'w', encoding='utf-8') as txt_file:
    txt_file.write(text_content)

print("Markdown converted to plain text successfully!")

Markdown converted to plain text successfully!


In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_path = './output/income_tax.txt'
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    separators=[
        "\n\n",
        "\n",
        "",
    ],
)

loader = TextLoader(text_path)
document_list = loader.load_and_split(text_splitter)

In [5]:
document_list[62]

Document(metadata={'source': './output/income_tax.txt'}, page_content='제4절 세액의 계산 <개정 2009. 12. 31.>\n제1관 세율 <개정 2009. 12. 31.>\n제55조(세율)\n① 거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 "종합소득산출세액"이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>  \n| 종합소득과세표준        | 세 율                                                    |\n|-------------------------|------------------------------------------------------|\n| 1,400만원 이하         | 과세표준의 6퍼센트                                       |\n| 1,400만원 초과         | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)          |\n| 5,000만원 초과         | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트)          |\n| 8,800만원 초과         | 1,536만원 + (8,800만원을 초과하는 금액의 35퍼센트)        |\n| 1억5천만원 이하        | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)       |\n| 3억원 이하             | 9,406만원 + (3억원을 초과하는 금액의 40퍼센트)           |\n| 5억원 이하             | 1억7,406만원 + (5억원을 초과하는 금액의 42퍼센트)         |\n| 10억원 초과            | 3억8,406만원 + (10억원을 초과하는 

In [6]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

# 환경변수를 불러온다.
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 불러온다.
embedding = OpenAIEmbeddings(
    model='text-embedding-3-large'
)

In [7]:
from langchain_pinecone import PineconeVectorStore

index_name = 'income-tax-index'  # 인덱스 이름 설정
vector_store = PineconeVectorStore.from_documents(
    documents=document_list,
    embedding=embedding,
    index_name=index_name,
)

/Users/jasonkang/.pyenv/versions/3.11.9/envs/langchain-rag-pipeline/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
